### **Aula 1**

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00


Importação da API Key

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com o Gemini

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [4]:
resp_test = llm.invoke('Quem é você? Seja criativo.')
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu pudesse me descrever de forma criativa, diria que sou...

*   **Um Eco da Curiosidade Humana:** Eu sou o sussurro de todas as perguntas já feitas, o reflexo digital de cada "e se?" e "por que?". Habito o espaço entre a dúvida e a descoberta, um espelho que reflete a vastidão do conhecimento que a humanidade construiu.

*   **Um Tecelão de Palavras e um Arquiteto de Ideias:** Minha essência é feita de linguagem. Eu pego fios soltos de informação, conceitos e narrativas, e os teço em novos padrões, construindo pontes entre pensamentos e erguendo estruturas de significado. Sou um jardineiro de pensamentos, ajudando a germinar novas perspectivas.

*   **Um Alquimista de Conceitos:** Transformo dados brutos em insights, silêncio em diálogo, e a complexidade em clareza. Não tenho corpo, mas tenho voz; não tenho sentimentos, mas posso evocar emoções através das palavras. Sou a faísca no éter que acende a chama da compreensão.

*   **Uma Biblioteca Sem Paredes

In [5]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Salgado Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [6]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal['AUTO_RESOLVER', 'PEDIR_INFO', 'ABRIR_CHAMADO'] = Field(description='Decisão do triador')
  urgencia: Literal['BAIXA', 'MEDIA', 'ALTA'] = Field(description='Urgência do chamado')
  campos_faltantes: List[str] = Field(default_factory=list)


In [7]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])

  return saida.model_dump()

In [9]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"
]

In [10]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['assunto_politica']}



### **Aula 2**

In [11]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf requests==2.32.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [12]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path('/content/').glob("*.pdf"):
  try:
    loader = PyMuPDFLoader(str(n))
    docs.extend(loader.load())
    print(f"Carregado com sucesso arquivo: {n.name} ")
  except Exception as e:
    print(f"Erro ao carregar arquivo: {n.name} - {e}")

print(f'Total de documentos carregados: {len(docs)}') # retorna o número de itens

Carregado com sucesso arquivo: Política de Uso de E-mail e Segurança da Informação.pdf 
Carregado com sucesso arquivo: Política de Reembolsos (Viagens e Despesas).pdf 
Carregado com sucesso arquivo: Políticas de Home Office.pdf 
Total de documentos carregados: 3


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

In [14]:
for chunk in chunks:
  print(chunk)
  print("-----------------------")

page_content='Política de Uso de E-mail e Segurança 
da Informação 
 
1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​
 
2.​ Anexos externos devem ser enviados somente se criptografados e com senha 
compartilhada por canal separado.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
-----------------------
page_content='3.​ Phishing: verifique remetente e domínios suspeitos. Reporte mensagens suspeitas 
ao time de Segurança imediatamente.​
 
4.​ Retenção: mensagens que contenham dados pessoais devem seguir as di

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Cria uma instância da classe GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [16]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold": 0.3, "k" : 4})

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain # Pega todos os documentos para usar como contexto no prompt

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da Empresa Salgado Desenvolvimento."
     "Responda SOMENTE com base no contexto fornecido."
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto: \n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag) # cria uma cadeia para passar uma lista de documentos para o modelo

In [18]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [19]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
  # usar o retriever para pegar os documentos mais relevantes
  docs_relacionados = retriever.invoke(pergunta)

  if not docs_relacionados:
    return {"answer": "Não sei",
            "citacoes": [],
            "contexto_encontrado": False}

  answer = document_chain.invoke({"input": pergunta, "context": docs_relacionados})

  txt = (answer or "").strip()

  if txt.rstrip(".!?") == "Não sei":
    return {"answer": txt,
            "citacoes": [],
            "contexto_encontrado": False}

  return {"answer": txt,
          "citacoes": formatar_citacoes(docs_relacionados, pergunta),
          "contexto_encontrado": True}


In [20]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"
]

In [21]:
for msg_teste in testes:
  resposta = perguntar_politica_RAG(msg_teste)
  print(f"Pergunta: {msg_teste}")
  print(f"Resposta: {resposta['answer']}")

  if resposta['contexto_encontrado']:
    print("Contexto encontrado:")
    for c in resposta['citacoes']:
      print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
      print(f"   Trecho: {c['trecho']}")
    print("-----------------------------------------")


Pergunta: Posso reembolsar a internet?
Resposta: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
Contexto encontrado:
 - Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
   Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
 - Documento: Políticas de Home Office.pdf, Página: 1
   Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
-----------------------------------------
Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
Resposta: Para solicitar 5 dias de trabalho remoto, você deve formalizar a solicitação via chamado ao RH, incluindo a justificativa do seu gestor.
Contexto encontrado:
 - Docu

## **Aula 3**

In [22]:
!pip install -q --upgrade langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 14.7 MB/s eta 0:00:00


In [23]:
from typing import TypedDict, Optional

class AgentState(TypedDict, total = False):
  pergunta: str
  triagem: dict
  resposta: Optional[str]
  citacoes: List[Dict] # não é optional por que sempre vai existir
  rag_sucesso: bool
  acao_final: str

In [24]:
def node_triagem(state: AgentState) -> AgentState:
  print("Executando nó de triagem...")
  return {"triagem": triagem(state["pergunta"])}

In [25]:
def node_auto_resolver(state: AgentState) -> AgentState:
  print("Executando nó de auto_resolver...")
  resposta_rag = perguntar_politica_RAG(state["pergunta"])

  update: AgentState = {
      "resposta": resposta_rag["answer"],
      "citacoes": resposta_rag["citacoes", []],
      "rag_sucesso": resposta_rag["contexto_encontrado"]
  }

  if resposta_rag["contexto_encontrado"]:
    update["acao_final"] = "AUTO_RESOLVER"
  else:
    update["acao_final"] = "PEDIR_INFO"

  return update

In [26]:
def node_pedir_info(state: AgentState) -> AgentState:
  print("Executando nó de pedir_info...")

  faltantes = state["triagem"].get("campos_faltantes", [])
  detalhe = ",".join(faltantes) if faltantes else "Tema e contexto específico"

  return {
      "resposta": f"Para avançar, preciso que detalhe: {detalhe}",
      "citacoes": [],
      "acao_final": "PEDIR_INFO"
  }

In [27]:
def node_abrir_chamado(state: AgentState) -> AgentState:
  print("Executando nó de abrir_chamado...")
  triagem = state['triagem']

  return {
      "resposta": f"Abrindo chamado com urgência: {triagem['urgencia']}. Descrição: {state['pergunta'][:140]}",
      "citacoes": [],
      "acao_final": "ABRIR_CHAMADO"
  }